In [1]:
import pandas as pd
import pickle
import numpy as np
import dgl
from dgl.data import DGLDataset
import torch
from tqdm import tqdm

def one_hot_encoding(n, n_to_index):
    one_hot_vector = [0]*(len(n_to_index))
    index = n_to_index[n]
    one_hot_vector[index] = 1
    return one_hot_vector

courses = pd.read_csv('data/archive/courses.csv')
vle = pd.read_csv('data/archive/vle.csv')
studentVle = pd.read_csv('data/archive/studentVle.csv')
studentRegistration = pd.read_csv('data/archive/studentRegistration.csv')
studentAssessment = pd.read_csv('data/archive/studentAssessment.csv')
studentInfo = pd.read_csv('data/archive/studentInfo.csv')
assessments = pd.read_csv('data/archive/assessments.csv')


/home/roh/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# learning_sequence에 이미 들어 있는 activity를 제외하고 다음 activity를 추가함
def add_learning_sequence(num_next_learning, learning_sequence):
    for i in range(len(num_next_learning)):
        if num_next_learning[i][0] in learning_sequence:
            pass
        else:
            learning_sequence.append(num_next_learning[i][0])
            break
    return learning_sequence, num_next_learning[i][0]

# list에 있는 activity들의 개수를 count함
def count_activity(first_learning):
    num_activity = {}
    for activity in list(set(first_learning)):
        num_activity[activity] = first_learning.count(activity)
    num_activity = sorted(num_activity.items(),reverse=True, key=lambda x:x[1])
    return num_activity

# next_learning이라는 리스트에 우리의 target_activity 다음에 오는 activity들을 추가함
def find_next_learning(student_list, target_activity, student_learning):
    next_learning = []
    for student in student_list:
        first_activity_index = [i for i in range(len(student_learning[student])) if target_activity == student_learning[student][i]]
        for index in first_activity_index:
            if index+1 >= len(student_learning[student]):
                pass
            else:
                next_learning.append(student_learning[student][index+1])
    return next_learning

studentInfo['completion_status'] = list(map(lambda x: 'Completion' if (x == 'Pass') or (x == 'Distinction') else 'Dropout', studentInfo['final_result']))
studentInfo['course_name'] = studentInfo['code_module'] + '_' + studentInfo['code_presentation']
vle['course_name'] = vle['code_module'] + '_' + vle['code_presentation']
studentVle['course_name'] = studentVle['code_module'] + '_' + studentVle['code_presentation']

course_learning_sequence = {}
learning_sequence = []

studentVle = studentVle[studentVle['date'] < 30]

for course in tqdm(list(set(vle['course_name']))):
    target_course = studentVle[studentVle['course_name'] == course]

    target_course_student = set(target_course['id_student'].to_list())
    target_course_activity = set(target_course['id_site'].to_list())

    student_learning = {}
    first_learning = []

    # 학생들 별로 learning sequence 담기
    for student in target_course_student:
        learing_list = target_course[target_course['id_student'] == student]['id_site'].to_list()
        student_learning[student] = learing_list
        first_learning.append(learing_list[0])

    # 첫번째 시작 activity 계산
    num_activity = count_activity(first_learning)
    target_activity = num_activity[0][0]
    learning_sequence.append(target_activity)

    for i in range(len(target_course_activity)):
        bf_ls_len = len(learning_sequence)
        next_learning = find_next_learning(target_course_student, target_activity, student_learning)
        num_next_learning = count_activity(next_learning)
        learning_sequence, target_activity = add_learning_sequence(num_next_learning, learning_sequence)
        if bf_ls_len == len(learning_sequence):
            break
    
    course_learning_sequence[course] = learning_sequence

100%|██████████| 22/22 [00:26<00:00,  1.22s/it]


In [3]:
# Graph build 순서
# course node 만들기 -> activity node 만들기 -> activity node와 course node 사이에 edge 생성
# activity node 사이에 edge만들기
# student node만들기 -> activity node와 edge 생성 -> course node와 edge 생성
studentVle = studentVle[studentVle['date'] >= 0]

node_set = {}
index = 0
course_node = list(set(vle['course_name']))
activity_node = list(set(vle['id_site']))
student_node = list(set(studentInfo['id_student']))
for i in course_node + activity_node + student_node:
    node_set[i] = index
    index += 1
    
# course node 만들기 -> activity node 만들기 -> activity node와 course node 사이에 edge 생성
src_node, dst_node = [], []
for i in range(len(vle)):
    src_node.append(node_set[vle['course_name'].to_numpy()[i]])
    dst_node.append(node_set[vle['id_site'].to_numpy()[i]])
    
# student node만들기 -> activity node와 edge 생성 
for i in range(len(studentVle)):
    src_node.append(node_set[studentVle['id_student'].to_numpy()[i]])
    dst_node.append(node_set[studentVle['id_site'].to_numpy()[i]])
    
# course node와 edge 생성 completion한 사람만 edge 생성
for i in range(len(studentInfo)):
    if studentInfo['completion_status'][i] == 'Completion':
        src_node.append(node_set[studentInfo['id_student'].to_numpy()[i]])
        dst_node.append(node_set[studentInfo['course_name'].to_numpy()[i]])

# activity 사이에 edge 생성
for course in list(set(vle['course_name'])):
    for i in range(len(course_learning_sequence[course])):
        src_node.append(node_set[course])
        dst_node.append(node_set[course_learning_sequence[course][i]])


In [4]:
g = dgl.graph((src_node, dst_node))

In [152]:
node_feature = []

# add node type feature
for i in range(len(course_node + dent_node)):


# date one-hot
date_feature = []
for student in student_node:
    base_date_feature = [0]*30
    student_date = list(set(studentVle[studentVle['id_student'] == student]['date']))
    for date in student_date:
        base_date_feature[date] = 1
    date_feature.append(base_date_feature)

,id_site,code_module,code_presentation,activity_type,week_from,week_to,course_name
0,546943,AAA,2013J,resource,NaN,NaN,AAA_2013J
1,546712,AAA,2013J,oucontent,NaN,NaN,AAA_2013J
2,546998,AAA,2013J,resource,NaN,NaN,AAA_2013J
3,546888,AAA,2013J,url,NaN,NaN,AAA_2013J
4,547035,AAA,2013J,resource,NaN,NaN,AAA_2013J
...,...,...,...,...,...,...,...
6359,897063,GGG,2014J,resource,NaN,NaN,GGG_2014J
6360,897109,GGG,2014J,resource,NaN,NaN,GGG_2014J
6361,896965,GGG,2014J,oucontent,NaN,NaN,GGG_2014J
6362,897060,GGG,2014J,resource,NaN,NaN,GGG_2014J


In [155]:
# make one-hot vector using activity_type, assessment_type
activity_onehot_list = list(vle['activity_type'].unique())
assessment_onehot_list = list(studentAssessment['assessment_type'].unique())

activity_zero_list = [0 for i in range(len(activity_onehot_list))]
assessment_zero_list = [0 for i in range(len(assessment_onehot_list))]

activity_to_index = {n : index for index, n in enumerate(activity_onehot_list)}
assessment_to_index = {n : index for index, n in enumerate(assessment_onehot_list)}

# activity one-hot
activity_node_feature, assessment_node_feature = [], []
studentVle = pd.merge(studentVle, vle[['id_site', 'activity_type']], left_on='id_activity', right_on='id_site')
for index, activity_type in enumerate(studentVle['activity_type']):
    activity_node_feature.append(one_hot_encoding(activity_type, activity_to_index))
    assessment_node_feature.append(assessment_zero_list)
studentVle['activity_node_feature'] = activity_node_feature
studentVle['assessment_node_feature'] = assessment_node_feature

# assessment one-hot
activity_node_feature, assessment_node_feature = [], []
for index, assessment_type in enumerate(studentAssessment['assessment_type']):
    activity_node_feature.append(activity_zero_list)
    assessment_node_feature.append(one_hot_encoding(assessment_type, assessment_to_index))
studentAssessment['activity_node_feature'] = activity_node_feature
studentAssessment['assessment_node_feature'] = assessment_node_feature

KeyError: 'id_activity'

In [157]:
vle

,id_site,code_module,code_presentation,activity_type,week_from,week_to,course_name
0,546943,AAA,2013J,resource,NaN,NaN,AAA_2013J
1,546712,AAA,2013J,oucontent,NaN,NaN,AAA_2013J
2,546998,AAA,2013J,resource,NaN,NaN,AAA_2013J
3,546888,AAA,2013J,url,NaN,NaN,AAA_2013J
4,547035,AAA,2013J,resource,NaN,NaN,AAA_2013J
...,...,...,...,...,...,...,...
6359,897063,GGG,2014J,resource,NaN,NaN,GGG_2014J
6360,897109,GGG,2014J,resource,NaN,NaN,GGG_2014J
6361,896965,GGG,2014J,oucontent,NaN,NaN,GGG_2014J
6362,897060,GGG,2014J,resource,NaN,NaN,GGG_2014J
